In [1]:
import pandas as pd
import numpy as np
import torch
import numpy as np
from torch_geometric.utils import dense_to_sparse
from tsl.data import SpatioTemporalDataset, SpatioTemporalDataModule
from tsl.data.preprocessing import StandardScaler
from tsl.data.datamodule.splitters import TemporalSplitter
import pytorch_lightning as pl
from tsl.engines import Predictor
from tsl.metrics.torch import MaskedMAE, MaskedMAPE, MaskedMSE
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import torch
import torch.nn as nn
from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange
from scipy.spatial.distance import pdist, squareform
import random
from pytorch_lightning import seed_everything

import warnings
warnings.filterwarnings('ignore')

print("PIPELINE: MODELADO GNN - KYLE FIELD")



c:\Users\DIEGO\OneDrive\Escritorio\Maestria Ciencia de datos\Tesis\Tests\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PIPELINE: MODELADO GNN - KYLE FIELD


In [3]:

def set_seed(seed=42):
    """Fijar semilla para reproducibilidad"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(seed, workers=True)

set_seed(42)

SEED = 42

random.seed(SEED)              
np.random.seed(SEED)          
torch.manual_seed(SEED)       
torch.cuda.manual_seed(SEED)  
torch.cuda.manual_seed_all(SEED)  

# PyTorch Lightning
seed_everything(SEED, workers=True)

# Configuración adicional para reproducibilidad
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"Semilla fijada: {SEED}")

Seed set to 42
Seed set to 42


Semilla fijada: 42


In [4]:
import torch

# 1. Cargar CSVs limpios (escala original)
train_df = pd.read_csv('train_estratificado.csv')
val_df = pd.read_csv('val_estratificado.csv')
test_df = pd.read_csv('test_estratificado.csv')


# Convertir fechas
for df in [train_df, val_df, test_df]:
    df['Date'] = pd.to_datetime(df['Date'])


In [8]:
# ============================================================================
# 2. APLICAR LOG-TRANSFORM A VARIABLES ASIMÉTRICAS
# ============================================================================
print("\nAplicar log-transform a variables asimétricas")

# Variables de producción (altamente asimétricas)
production_cols = ['Oil (m3)', 'Gas (m3)', 'Produced Water (m3)']

for df in [train_df, val_df, test_df]:
    # Asegurar valores no negativos
    df[production_cols] = df[production_cols].clip(lower=0)
    
    # Aplicar log1p (log(1+x) para manejar ceros)
    df[production_cols] = np.log1p(df[production_cols])

print("Log-transform aplicado a producción")

# Verificar rangos después de log
print(f"\n Rangos después de log-transform (train):")
print(f"  Oil range:   [{train_df['Oil (m3)'].min():.2f}, {train_df['Oil (m3)'].max():.2f}]")
print(f"  Gas range:   [{train_df['Gas (m3)'].min():.2f}, {train_df['Gas (m3)'].max():.2f}]")
print(f"  Water range: [{train_df['Produced Water (m3)'].min():.2f}, {train_df['Produced Water (m3)'].max():.2f}]")

# Verificar distribuciones por pozo
print(f"\n Estadísticas por pozo (Gas log):")
for well in ['Well_12', 'Well_13', 'Well_14']:
    gas_mean = train_df[train_df['Well_ID'] == well]['Gas (m3)'].mean()
    gas_std = train_df[train_df['Well_ID'] == well]['Gas (m3)'].std()
    print(f"  {well}: mean={gas_mean:.2f}, std={gas_std:.2f}")


Aplicar log-transform a variables asimétricas
Log-transform aplicado a producción

 Rangos después de log-transform (train):
  Oil range:   [0.00, 2.22]
  Gas range:   [0.00, 2.71]
  Water range: [0.00, 2.24]

 Estadísticas por pozo (Gas log):
  Well_12: mean=2.51, std=0.44
  Well_13: mean=2.45, std=0.51
  Well_14: mean=2.47, std=0.12


In [9]:
# ============================================================================
# 3. DEFINIR FEATURES Y WELL MAPPING (3 POZOS)
# ============================================================================
print("\nDefinir features y mapping")

feature_cols = [
    'Hours Online', 
    'Av. WHT (Deg C)', 
    'Av. WHP (bar)', 
    'Av. DHT (Deg C)', 
    'Av. DHP (bar)', 
    'Oil (m3)',              
    'Gas (m3)',              
    'Produced Water (m3)'    
]

# 3 pozos
well_mapping = {
    'Well_12': 0, 
    'Well_13': 1, 
    'Well_14': 2
}

print(f"Features: {len(feature_cols)}")
print(f"Pozos: {len(well_mapping)}")


Definir features y mapping
Features: 8
Pozos: 3


In [10]:
# ============================================================================
# 4. FUNCIÓN PARA CREAR TENSORES
# ============================================================================
def create_tensor_from_df(df, feature_cols, well_mapping):
    """
    Crea tensor [T, N, F] desde DataFrame
    T = timesteps, N = nodos, F = features
    """
    # Ordenar por fecha
    df = df.sort_values('Date').reset_index(drop=True)
    
    # Obtener fechas únicas ordenadas
    dates = sorted(df['Date'].unique())
    n_times = len(dates)
    n_nodes = len(well_mapping)
    n_features = len(feature_cols)
    
    # Inicializar tensor con ceros
    tensor = torch.zeros((n_times, n_nodes, n_features), dtype=torch.float32)
    
    print(f"  Creando tensor: {n_times} timesteps × {n_nodes} nodos × {n_features} features")
    
    # Llenar tensor fecha por fecha
    for date_idx, date in enumerate(dates):
        day_data = df[df['Date'] == date]
        
        # Para cada pozo en esa fecha
        for well_id, well_idx in well_mapping.items():
            well_day_data = day_data[day_data['Well_ID'] == well_id]
            
            if len(well_day_data) > 0:
                values = well_day_data[feature_cols].iloc[0].values
                tensor[date_idx, well_idx, :] = torch.tensor(values, dtype=torch.float32)
    
    return tensor

In [11]:
# ============================================================================
# 5. CREAR TENSORES SEPARADOS
# ============================================================================
print("\nCrear tensores para cada conjunto")

train_tensor = create_tensor_from_df(train_df, feature_cols, well_mapping)
val_tensor = create_tensor_from_df(val_df, feature_cols, well_mapping)
test_tensor = create_tensor_from_df(test_df, feature_cols, well_mapping)

print(f"\nTrain tensor: {train_tensor.shape}")
print(f"Val tensor:   {val_tensor.shape}")
print(f"Test tensor:  {test_tensor.shape}")

# Crear tensores TARGET (solo 3 features de producción)
target_cols = ['Oil (m3)', 'Gas (m3)', 'Produced Water (m3)']

train_target = create_tensor_from_df(train_df, target_cols, well_mapping)
val_target = create_tensor_from_df(val_df, target_cols, well_mapping)
test_target = create_tensor_from_df(test_df, target_cols, well_mapping)

print(f"\nTensores TARGET (3 features):")
print(f"  Train: {train_target.shape}")
print(f"  Val:   {val_target.shape}")
print(f"  Test:  {test_target.shape}")


Crear tensores para cada conjunto
  Creando tensor: 4034 timesteps × 3 nodos × 8 features
  Creando tensor: 1444 timesteps × 3 nodos × 8 features
  Creando tensor: 1336 timesteps × 3 nodos × 8 features

Train tensor: torch.Size([4034, 3, 8])
Val tensor:   torch.Size([1444, 3, 8])
Test tensor:  torch.Size([1336, 3, 8])
  Creando tensor: 4034 timesteps × 3 nodos × 3 features
  Creando tensor: 1444 timesteps × 3 nodos × 3 features
  Creando tensor: 1336 timesteps × 3 nodos × 3 features

Tensores TARGET (3 features):
  Train: torch.Size([4034, 3, 3])
  Val:   torch.Size([1444, 3, 3])
  Test:  torch.Size([1336, 3, 3])


In [13]:
# ============================================================
# 6: CREAR GRAFO ESPACIAL
# ============================================================
print("\n" + "="*70)
print("CREAR GRAFO ESPACIAL")
print("="*70)

coords = {
    'Well_12': [1.23349, 56.8859],
    'Well_13': [1.22352, 56.9148], 
    'Well_14': [1.22292, 56.9146]
}

coords_array = np.array([coords[f'Well_{i}'] for i in [12, 13, 14]])
distances = squareform(pdist(coords_array, metric='euclidean'))

# Convertir a pesos
epsilon = 1e-5
adj_matrix = 1 / (distances + epsilon)
np.fill_diagonal(adj_matrix, 0)
adj_matrix = adj_matrix / adj_matrix.max()

print(f"Matriz de adyacencia:\n{adj_matrix.round(4)}")

# Crear edge_index y edge_weight
src_nodes = []
dst_nodes = []
weights = []

for i in range(3):
    for j in range(3):
        if i != j:
            src_nodes.append(i)
            dst_nodes.append(j)
            weights.append(adj_matrix[i, j])

edge_index = torch.tensor([src_nodes, dst_nodes], dtype=torch.long)
edge_weight = torch.tensor(weights, dtype=torch.float32)

print(f"\nEdge index:  {edge_index.shape}")
print(f"Edge weight: {edge_weight.shape}")
print(f"Número de edges: {len(src_nodes)}")


CREAR GRAFO ESPACIAL
Matriz de adyacencia:
[[0.    0.021 0.021]
 [0.021 0.    1.   ]
 [0.021 1.    0.   ]]

Edge index:  torch.Size([2, 6])
Edge weight: torch.Size([6])
Número de edges: 6


In [14]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# ============================================================
# 7: CREAR Y AJUSTAR SCALERS (SOLO CON TRAIN)
# ============================================================
print("\n" + "="*70)
print("CREAR SCALERS CON DATOS DE TRAIN")
print("="*70)

# Scalers para INPUT (8 features)
print("\nScalers INPUT (8 features):")
x_scalers = []

for feat_idx in range(8):
    scaler = StandardScaler()
    train_feat = train_tensor[:, :, feat_idx].numpy()
    train_feat_flat = train_feat.reshape(-1, 1)
    scaler.fit(train_feat_flat)
    x_scalers.append(scaler)
    
    feat_name = feature_cols[feat_idx]
    print(f"  [{feat_idx}] {feat_name:25s}: mean={scaler.mean_[0]:8.4f}, std={scaler.scale_[0]:8.4f}")

# Scalers para TARGET (3 features)
print("\nScalers TARGET (3 features):")
y_scalers = []

for feat_idx in range(3):
    scaler = StandardScaler()
    train_target_feat = train_target[:, :, feat_idx].numpy()
    train_target_flat = train_target_feat.reshape(-1, 1)
    scaler.fit(train_target_flat)
    y_scalers.append(scaler)
    
    print(f"  [{feat_idx}] {target_cols[feat_idx]:25s}: mean={scaler.mean_[0]:8.4f}, std={scaler.scale_[0]:8.4f}")

print(f"\n{len(x_scalers)} scalers INPUT creados")
print(f"{len(y_scalers)} scalers TARGET creados")


CREAR SCALERS CON DATOS DE TRAIN

Scalers INPUT (8 features):
  [0] Hours Online             : mean= 16.1953, std= 10.8502
  [1] Av. WHT (Deg C)          : mean= 45.1368, std= 31.8013
  [2] Av. WHP (bar)            : mean= 42.2300, std= 34.9593
  [3] Av. DHT (Deg C)          : mean= 67.0081, std= 43.3415
  [4] Av. DHP (bar)            : mean= 83.0250, std= 60.6683
  [5] Oil (m3)                 : mean=  1.3311, std=  0.8753
  [6] Gas (m3)                 : mean=  1.7619, std=  1.1678
  [7] Produced Water (m3)      : mean=  0.4984, std=  0.7464

Scalers TARGET (3 features):
  [0] Oil (m3)                 : mean=  1.3311, std=  0.8753
  [1] Gas (m3)                 : mean=  1.7619, std=  1.1678
  [2] Produced Water (m3)      : mean=  0.4984, std=  0.7464

8 scalers INPUT creados
3 scalers TARGET creados


In [15]:
# ============================================================
# 8: NORMALIZAR DATOS
# ============================================================
print("\n" + "="*70)
print("NORMALIZAR DATOS")
print("="*70)

def normalize_tensor(tensor, scalers):
    """Normaliza tensor [T, N, F] con lista de scalers"""
    tensor_norm = tensor.clone()
    T, N, F = tensor.shape
    
    for feat_idx in range(F):
        feat_data = tensor[:, :, feat_idx].numpy()
        feat_flat = feat_data.reshape(-1, 1)
        feat_norm = scalers[feat_idx].transform(feat_flat)
        feat_reshaped = feat_norm.reshape(T, N)
        tensor_norm[:, :, feat_idx] = torch.from_numpy(feat_reshaped)
    
    return tensor_norm

# Normalizar INPUT
train_tensor_norm = normalize_tensor(train_tensor, x_scalers)
val_tensor_norm = normalize_tensor(val_tensor, x_scalers)
test_tensor_norm = normalize_tensor(test_tensor, x_scalers)

# Normalizar TARGET
train_target_norm = normalize_tensor(train_target, y_scalers)
val_target_norm = normalize_tensor(val_target, y_scalers)
test_target_norm = normalize_tensor(test_target, y_scalers)

print(f"INPUT normalizado:")
print(f"  Train: {train_tensor_norm.shape}")
print(f"  Val:   {val_tensor_norm.shape}")
print(f"  Test:  {test_tensor_norm.shape}")

print(f"\nTARGET normalizado:")
print(f"  Train: {train_target_norm.shape}")
print(f"  Val:   {val_target_norm.shape}")
print(f"  Test:  {test_target_norm.shape}")


NORMALIZAR DATOS
INPUT normalizado:
  Train: torch.Size([4034, 3, 8])
  Val:   torch.Size([1444, 3, 8])
  Test:  torch.Size([1336, 3, 8])

TARGET normalizado:
  Train: torch.Size([4034, 3, 3])
  Val:   torch.Size([1444, 3, 3])
  Test:  torch.Size([1336, 3, 3])


In [17]:
# ============================================================
# 9: CONCATENAR PARA DATASET ÚNICO 
# ============================================================
print("\n" + "="*70)
print("CONCATENAR TENSORES MANTENIENDO ORDEN")
print("="*70)

# CLAVE: Concatenar DESPUÉS de normalizar, ANTES de crear el dataset
full_input_norm = torch.cat([train_tensor_norm, val_tensor_norm, test_tensor_norm], dim=0)
full_target_norm = torch.cat([train_target_norm, val_target_norm, test_target_norm], dim=0)

print(f"full_input_norm:  {full_input_norm.shape}")
print(f"full_target_norm: {full_target_norm.shape}")

# Guardar longitudes para el split
train_length = len(train_tensor_norm)
val_length = len(val_tensor_norm)
test_length = len(test_tensor_norm)

print(f"\nLongitudes de tensores:")
print(f"  Train: {train_length} timesteps")
print(f"  Val:   {val_length} timesteps")
print(f"  Test:  {test_length} timesteps")


CONCATENAR TENSORES MANTENIENDO ORDEN
full_input_norm:  torch.Size([6814, 3, 8])
full_target_norm: torch.Size([6814, 3, 3])

Longitudes de tensores:
  Train: 4034 timesteps
  Val:   1444 timesteps
  Test:  1336 timesteps


In [19]:
# ============================================================
# 10: CREAR UN ÚNICO DATASET
# ============================================================
print("\n" + "="*70)
print("CREAR UN ÚNICO SPATIOTEMPORALDATASET")
print("="*70)

window = 14
horizon = 7
stride = 1


torch_dataset_norm = SpatioTemporalDataset(
    target=full_target_norm.numpy(),
    covariates={'x': full_input_norm.numpy()},
    connectivity=(edge_index, edge_weight),
    horizon=horizon,
    window=window,
    stride=stride
)

print(f"Dataset creado: {len(torch_dataset_norm)} samples")


# Calcular tamaños esperados
total_length = train_length + val_length + test_length
n_total_samples = total_length - window - horizon + 1

print(f"\nDataset esperado:")
print(f"  Total timesteps: {total_length}")
print(f"  Total samples: {n_total_samples}")
print(f"  Window: {window}, Horizon: {horizon}, Stride: {stride}")


CREAR UN ÚNICO SPATIOTEMPORALDATASET
Dataset creado: 6794 samples

Dataset esperado:
  Total timesteps: 6814
  Total samples: 6794
  Window: 14, Horizon: 7, Stride: 1


In [21]:
# ============================================================
# 11: CALCULAR ÍNDICES DE SPLIT CORRECTOS
# ============================================================
print("\n" + "="*70)
print("CALCULAR ÍNDICES DE SPLIT")
print("="*70)

# Calcular cuántos samples genera cada segmento
n_train_samples = train_length - window - horizon + 1
n_val_samples = val_length - window - horizon + 1
n_test_samples = test_length - window - horizon + 1

print(f"\nSamples por segmento:")
print(f"  Train: {n_train_samples} samples (timesteps 0-{train_length-1})")
print(f"  Val:   {n_val_samples} samples (timesteps {train_length}-{train_length+val_length-1})")
print(f"  Test:  {n_test_samples} samples (timesteps {train_length+val_length}-{total_length-1})")


train_end_idx = n_train_samples

val_start_idx = train_end_idx
val_end_idx = val_start_idx + n_val_samples

# Test: empieza donde termina val
test_start_idx = val_end_idx
test_end_idx = test_start_idx + n_test_samples

train_idxs = list(range(0, train_end_idx))
val_idxs = list(range(val_start_idx, val_end_idx))
test_idxs = list(range(test_start_idx, test_end_idx))

print(f"\nÍndices de split:")
print(f"  Train: {0} → {train_end_idx-1} ({len(train_idxs)} samples)")
print(f"  Val:   {val_start_idx} → {val_end_idx-1} ({len(val_idxs)} samples)")
print(f"  Test:  {test_start_idx} → {test_end_idx-1} ({len(test_idxs)} samples)")
print(f"  Total: {len(train_idxs) + len(val_idxs) + len(test_idxs)} samples")

# Verificación
assert len(train_idxs) == n_train_samples, "❌ Train samples no coinciden"
assert len(val_idxs) == n_val_samples, "❌ Val samples no coinciden"
assert len(test_idxs) == n_test_samples, "❌ Test samples no coinciden"
print("\nVerificación de índices: CORRECTA")


CALCULAR ÍNDICES DE SPLIT

Samples por segmento:
  Train: 4014 samples (timesteps 0-4033)
  Val:   1424 samples (timesteps 4034-5477)
  Test:  1316 samples (timesteps 5478-6813)

Índices de split:
  Train: 0 → 4013 (4014 samples)
  Val:   4014 → 5437 (1424 samples)
  Test:  5438 → 6753 (1316 samples)
  Total: 6754 samples

Verificación de índices: CORRECTA


In [23]:
# ============================================================
# 12: CREAR SPLITTER Y DATAMODULE
# ============================================================
print("\n" + "="*70)
print("CREAR SPLITTER Y DATAMODULE")
print("="*70)

class ManualSplitter:
    """Splitter que respeta la estratificación temporal"""
    def __init__(self, train_idxs, val_idxs, test_idxs):
        self.train_idxs = train_idxs
        self.val_idxs = val_idxs
        self.test_idxs = test_idxs
    
    def split(self, dataset, **kwargs):
        return self.train_idxs, self.val_idxs, self.test_idxs

manual_splitter = ManualSplitter(train_idxs, val_idxs, test_idxs)
print("Splitter manual creado")


# Descomenta cuando tengas tsl disponible
from tsl.data import SpatioTemporalDataModule

# Fijar semilla antes de crear DataModule
torch.manual_seed(SEED)
np.random.seed(SEED)

dm_norm = SpatioTemporalDataModule(
    dataset=torch_dataset_norm,  # ✅ UN SOLO dataset, NO ConcatDataset
    scalers=None,  # Ya normalizados
    splitter=manual_splitter,
    batch_size=32,
    workers=0,
)

dm_norm.setup()

print("DataModule creado y configurado")



CREAR SPLITTER Y DATAMODULE
Splitter manual creado
DataModule creado y configurado


In [26]:
# ============================================================
# VERIFICACIÓN FINAL DE ESTRATIFICACIÓN
# ============================================================
print("\n" + "="*70)
print("VERIFICACIÓN FINAL")
print("="*70)

print("\nIMPORTANCIA DE ESTRATIFICACIÓN PRESERVADA:")
print("  1. ✓ Datos divididos por pozo (70/15/15%)")
print("  2. ✓ Normalización con scalers SOLO de train")
print("  3. ✓ Tensores concatenados EN ORDEN (train→val→test)")
print("  4. ✓ UN SOLO SpatioTemporalDataset (no ConcatDataset)")
print("  5. ✓ Índices calculados para respetar límites originales")
print("  6. ✓ ManualSplitter preserva la estratificación")





VERIFICACIÓN FINAL

IMPORTANCIA DE ESTRATIFICACIÓN PRESERVADA:
  1. ✓ Datos divididos por pozo (70/15/15%)
  2. ✓ Normalización con scalers SOLO de train
  3. ✓ Tensores concatenados EN ORDEN (train→val→test)
  4. ✓ UN SOLO SpatioTemporalDataset (no ConcatDataset)
  5. ✓ Índices calculados para respetar límites originales
  6. ✓ ManualSplitter preserva la estratificación


## TIMETHENSPACE MODEL

In [27]:

from einops import rearrange
from einops.layers.torch import Rearrange
# ============================================================================
# DEFINIR MODELO TIMETHENSPACE
# ============================================================================
class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 edge_index, edge_weight,
                 hidden_size: int = 96,
                 rnn_layers: int = 3,
                 gnn_kernel: int = 5,
                 dropout: float = 0.05,
                 output_size: int = 3):
        super(TimeThenSpaceModel, self).__init__()
        
        self.input_size = input_size      # 8 features de entrada
        self.output_size = output_size    # 3 features de salida (Oil, Gas, Water)
        self.horizon = horizon            # 7 días a predecir
        self.hidden_size = hidden_size
        self.n_nodes = n_nodes            # 4 pozos
        
        # Encoder: input_size → hidden_size
        self.encoder = nn.Linear(input_size, hidden_size)
        
        # Node embeddings
        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)
        
        # Módulo temporal (RNN)
        self.time_nn = RNN(
            input_size=hidden_size,
            hidden_size=hidden_size,
            n_layers=rnn_layers,
            cell='gru',
            return_only_last_state=True
        )
        
        # Módulo espacial (Graph Convolution)
        self.space_nn = DiffConv(
            in_channels=hidden_size,
            out_channels=hidden_size,
            k=gnn_kernel
        )
        
        # Decoder: hidden_size → output_size * horizon
        self.decoder = nn.Linear(hidden_size, output_size * horizon)
        
        # Rearrange: (batch, nodes, horizon*output) → (batch, horizon, nodes, output)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon, f=output_size)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Registrar grafo como buffers
        self.register_buffer('edge_index', edge_index)
        self.register_buffer('edge_weight', edge_weight)
    
    def forward(self, x):
        """
        Args:
            x: [batch, window, nodes, input_size] 
               Ejemplo: [32, 14, 4, 8]
        
        Returns:
            out: [batch, horizon, nodes, output_size]
                 Ejemplo: [32, 7, 4, 3]
        """
        # Encoder: [b, w, n, 8] → [b, w, n, hidden]
        x_enc = self.encoder(x)
        
        # Add node embeddings: [b, w, n, hidden]
        x_emb = x_enc + self.node_embeddings()
        
        # Temporal processing: [b, w, n, hidden] → [b, n, hidden]
        h = self.time_nn(x_emb)
        
        # Spatial processing: [b, n, hidden] → [b, n, hidden]
        z = self.space_nn(h, self.edge_index, self.edge_weight)
        
        # Dropout
        z = self.dropout(z)
        
        # Decoder: [b, n, hidden] → [b, n, horizon*output]
        # Ejemplo: [32, 4, 96] → [32, 4, 21]
        x_out = self.decoder(z)
        
        # Rearrange: [b, n, 21] → [b, 7, 4, 3]
        x_horizon = self.rearrange(x_out)
        
        return x_horizon


In [ ]:
# ============================================================================
# 5B. RECREAR EDGE_INDEX CORRECTAMENTE
# ============================================================================
src_nodes = []
dst_nodes = []
weights = []

for i in range(3):
    for j in range(3):
        if i != j:
            src_nodes.append(i)
            dst_nodes.append(j)
            weights.append(adj_matrix[i, j])

# Crear tensores
edge_index_tensor = torch.tensor([src_nodes, dst_nodes], dtype=torch.long)  # [2, 12]
edge_weight = torch.tensor(weights, dtype=torch.float32)  # [12]

print(f"\nEdge index tensor shape: {edge_index_tensor.shape}")  # [2, 12]
print(f"Edge weight shape: {edge_weight.shape}")  # [12]
print(f"Edge index:\n{edge_index_tensor}")
model = TimeThenSpaceModel(
    input_size=8,           # 8 features de entrada
    output_size=3,          # 3 features de salida (Oil, Gas, Water)
    n_nodes=3,              # 4 pozos
    horizon=7,              # Predecir 7 días
    edge_index=edge_index_tensor,       # Tensor de aristas [12, 2]
    edge_weight=edge_weight, # Pesos de las aristas
    hidden_size=128,
    rnn_layers=4,
    gnn_kernel=4,
)
print(model)
def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)
print_model_size(model)


Edge index tensor shape: torch.Size([2, 6])
Edge weight shape: torch.Size([6])
Edge index:
tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 0, 2, 0, 1]])
TimeThenSpaceModel(
  (encoder): Linear(in_features=8, out_features=128, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=3, embedding_size=128)
  (time_nn): RNN(
    (rnn): GRU(128, 128, num_layers=4)
  )
  (space_nn): DiffConv(128, 128)
  (decoder): Linear(in_features=128, out_features=21, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=7, f=3)
  (dropout): Dropout(p=0.05, inplace=False)
)
Number of model (TimeThenSpaceModel) parameters:    548117


In [ ]:
class MaskedR2(nn.Module):
    """R² Score with mask support"""
    
    def __init__(self):
        super().__init__()
    
    def forward(self, y_hat, y, mask=None):
        if mask is None:
            mask = torch.ones_like(y, dtype=torch.bool)
        
        y_hat_masked = y_hat[mask]
        y_masked = y[mask]
        
        ss_res = torch.sum((y_masked - y_hat_masked) ** 2)
        y_mean = torch.mean(y_masked)
        ss_tot = torch.sum((y_masked - y_mean) ** 2)
        
        if ss_tot == 0:
            return torch.tensor(0.0, device=y.device)
        
        r2 = 1 - (ss_res / ss_tot)
        return r2


# Crear predictor con métricas
predictor = Predictor(
    model=model,
    optim_class=torch.optim.AdamW,
    optim_kwargs={'lr': 0.00005, 'weight_decay': 0.0001},
    loss_fn=MaskedMAE(),
    metrics={
        'mae': MaskedMAE(),      
        'mse': MaskedMSE(),     
        'r2': MaskedR2()
    }
)


# Checkpoint optimizado para Kyle Field
checkpoint_callback = ModelCheckpoint(
    dirpath='logs/kyle_field',
    save_top_k=1,
    monitor='val_mae',              
    mode='min',
    filename='kyle-timespace-{epoch:02d}-{val_mae:.4f}',
    save_weights_only=False,
    verbose=True
)

# Early stopping para evitar overfitting
early_stopping = EarlyStopping(
    monitor='val_mae',              # Monitorear MAE en lugar de MAPE
    patience=15,                    
    min_delta=0.01,                
    mode='min',
    verbose=True
)

# Trainer optimizado para Kyle Field
trainer = pl.Trainer(
    max_epochs=100,                 
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    gradient_clip_val=1.0,         
    limit_train_batches=None,       
    val_check_interval=1.0,         
    log_every_n_steps=5,            
    enable_progress_bar=True,
    callbacks=[checkpoint_callback, early_stopping],
    deterministic=True,            
    benchmark=False                  
)

# Entrenamiento
print("Iniciando entrenamiento TimeThenSpace para Kyle Field...")
trainer.fit(predictor, datamodule=dm_norm)

# Evaluación final
test_results = trainer.test(predictor, datamodule=dm_norm)
print("\nResultados finales TimeThenSpace:")
for metric, value in test_results[0].items():
    if 'mape' not in metric.lower(): 
        print(f"{metric}: {value:.4f}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..

  | Name          | Type               | Params | Mode 
-------------------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0      | train
1 | train_metrics | MetricCollection   | 0      | train
2 | val_metrics   | MetricCollection   | 0      | train
3 | test_metrics  | MetricCollection   | 0      | train
4 | model         | TimeThenSpaceModel | 548 K  | train
-------------------------------------------------------------
548 K     Trainable params
0         Non-trainable params
548 K     Total params
2.192     Total estimated model params size (MB)
24        Modules in train mode
0         Modules in eval mode


Iniciando entrenamiento TimeThenSpace para Kyle Field...
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

Arguments ['edge_index', 'edge_weight'] are filtered out. Only args ['x'] are forwarded to the model (TimeThenSpaceModel).


Epoch 0: 100%|██████████| 125/125 [00:13<00:00,  9.34it/s, v_num=42, val_mae=0.518, val_mse=0.711, val_r2=0.388, train_mae=0.634, train_mse=0.836, train_r2=0.161]

Metric val_mae improved. New best score: 0.518
Epoch 0, global step 125: 'val_mae' reached 0.51799 (best 0.51799), saving model to 'C:\\Users\\DIEGO\\OneDrive\\Escritorio\\Maestria Ciencia de datos\\Tesis\\Tests\\mape copy\\logs\\kyle_field\\kyle-timespace-epoch=00-val_mae=0.5180.ckpt' as top 1


...

Epoch 27: 100%|██████████| 125/125 [00:14<00:00,  8.49it/s, v_num=42, val_mae=0.242, val_mse=0.383, val_r2=0.659, train_mae=0.217, train_mse=0.300, train_r2=0.698]

Epoch 27, global step 3500: 'val_mae' reached 0.24206 (best 0.24206), saving model to 'C:\\Users\\DIEGO\\OneDrive\\Escritorio\\Maestria Ciencia de datos\\Tesis\\Tests\\mape copy\\logs\\kyle_field\\kyle-timespace-epoch=27-val_mae=0.2421-v1.ckpt' as top 1


Epoch 28: 100%|██████████| 125/125 [00:14<00:00,  8.50it/s, v_num=42, val_mae=0.244, val_mse=0.383, val_r2=0.658, train_mae=0.216, train_mse=0.301, train_r2=0.698]

Monitored metric val_mae did not improve in the last 15 records. Best score: 0.248. Signaling Trainer to stop.
Epoch 28, global step 3625: 'val_mae' was not in top 1


Testing DataLoader 0: 100%|██████████| 42/42 [00:01<00:00, 27.01it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.09320951998233795
        test_mae            0.09320952743291855
        test_mse            0.07947178184986115
         test_r2             0.925626277923584
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Resultados finales TimeThenSpace:
test_mae: 0.0932
test_mse: 0.0795
test_r2: 0.9256
test_loss: 0.0932


In [ ]:
def evaluate_all_sets_manual(predictor, dm_norm, train_tensor, full_target, train_length, val_length):

    
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import r2_score
    
    print("="*80)
    print("EVALUACIÓN COMPLETA: TRAIN, VAL y TEST")
    print("="*80)
    
    # ========================================================================
    # RECREAR SCALERS
    # ========================================================================
    print("\n[0/5] Recreando scalers desde datos de train...")
    
    target_names = ['Oil (m3)', 'Gas (m3)', 'Produced Water (m3)']
    y_scalers = []
    
    # IMPORTANTE: Los scalers se ajustan SOLO con datos de train
    for feat_idx in range(3):
        scaler = StandardScaler()
        # full_target[:train_length] son los datos de TRAIN
        train_target_feat = full_target[:train_length, :, feat_idx].numpy()
        train_target_flat = train_target_feat.reshape(-1, 1)
        scaler.fit(train_target_flat)
        y_scalers.append(scaler)
        
        print(f"  Scaler {feat_idx} ({target_names[feat_idx]}): "
              f"mean={scaler.mean_[0]:.4f}, std={scaler.scale_[0]:.4f}")
    
    print("✓ Scalers recreados con datos de TRAIN")
    
    # ========================================================================
    # FUNCIÓN AUXILIAR: EVALUAR UN CONJUNTO
    # ========================================================================
    def evaluate_single_set(dataloader, set_name):
        """Evalúa un conjunto específico"""
        
        print(f"\n{'='*80}")
        print(f"EVALUANDO: {set_name}")
        print(f"{'='*80}")
        
        model = predictor.model
        model.eval()
        
        predictions_norm = []
        targets_norm = []
        
        print(f"\n[1/4] Obteniendo predicciones de {set_name}...")
        
        with torch.no_grad():
            for batch in dataloader:
                y_pred = model(batch.x)
                predictions_norm.append(y_pred.detach().cpu())
                targets_norm.append(batch.y.detach().cpu())
        
        predictions_norm = torch.cat(predictions_norm, dim=0)
        targets_norm = torch.cat(targets_norm, dim=0)
        
        print(f"✓ Predicciones: {predictions_norm.shape}")
        print(f"  Nota: Están en escala NORMALIZADA")
        
        # ====================================================================
        # DESNORMALIZAR
        # ====================================================================
        print(f"\n[2/4] Desnormalizando...")
        
        S, H, N, F = predictions_norm.shape
        pred_flat = predictions_norm.reshape(-1, F).numpy()
        target_flat = targets_norm.reshape(-1, F).numpy()
        
        pred_desnorm = np.zeros_like(pred_flat)
        target_desnorm = np.zeros_like(target_flat)
        
        # Desnormalizar cada feature
        for feat_idx in range(F):
            pred_desnorm[:, feat_idx] = y_scalers[feat_idx].inverse_transform(
                pred_flat[:, feat_idx].reshape(-1, 1)
            ).flatten()
            target_desnorm[:, feat_idx] = y_scalers[feat_idx].inverse_transform(
                target_flat[:, feat_idx].reshape(-1, 1)
            ).flatten()
        
        print(f"✓ Desnormalizado (aún con log): [{pred_desnorm.min():.2f}, {pred_desnorm.max():.2f}]")
        
        # ====================================================================
        # INVERTIR LOG-TRANSFORM
        # ====================================================================
        print(f"\n[3/4] Invirtiendo log-transform (np.expm1)...")
        
        # np.expm1(x) = exp(x) - 1, que invierte np.log1p(x)
        predictions_original = np.expm1(pred_desnorm)
        targets_original = np.expm1(target_desnorm)
        
        # Asegurar que no hay valores negativos
        predictions_original = np.maximum(predictions_original, 0)
        targets_original = np.maximum(targets_original, 0)
        
        print(f"✓ Escala original (m³): [{predictions_original.min():.2f}, {predictions_original.max():.2f}]")
        
        # Verificación de sanidad
        if targets_original.max() > 2e6:
            print(f"\n🚨 ERROR: Max target={targets_original.max():.2e} m³")
            print(f"   Esto es anormalmente alto, puede haber un error en la desnormalización")
            return None
        
        # ====================================================================
        # CALCULAR MÉTRICAS
        # ====================================================================
        print(f"\n[4/4] Calculando métricas...")
        
        epsilon = 1e-8
        
        # MAE (Mean Absolute Error)
        mae = np.mean(np.abs(targets_original - predictions_original))
        
        # MAPE (Mean Absolute Percentage Error) - evitar división por cero
        mask_mape = targets_original > 0.1
        mape = np.mean(np.abs((targets_original[mask_mape] - predictions_original[mask_mape]) / 
                       targets_original[mask_mape])) * 100 if mask_mape.sum() > 0 else 0
        
        # SMAPE (Symmetric Mean Absolute Percentage Error)
        smape = np.mean(2 * np.abs(targets_original - predictions_original) / 
                        (np.abs(targets_original) + np.abs(predictions_original) + epsilon)) * 100
        
        # WMAPE (Weighted Mean Absolute Percentage Error)
        wmape = (np.sum(np.abs(targets_original - predictions_original)) / 
                 (np.sum(np.abs(targets_original)) + epsilon)) * 100
        
        # MAPE con umbral (solo para valores > 10)
        mask_mape_10 = targets_original > 10
        mape_10 = np.mean(np.abs((targets_original[mask_mape_10] - predictions_original[mask_mape_10]) / 
                          targets_original[mask_mape_10])) * 100 if mask_mape_10.sum() > 0 else 0
        
        # RMSE (Root Mean Squared Error)
        rmse = np.sqrt(np.mean((targets_original - predictions_original) ** 2))
        
        # R² (Coefficient of Determination)
        ss_res = np.sum((targets_original - predictions_original) ** 2)
        ss_tot = np.sum((targets_original - np.mean(targets_original)) ** 2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        # Métricas para producción activa (> 10 m³)
        active_mask = targets_original > 10
        active_count = active_mask.sum()
        
        if active_count > 0:
            mae_active = np.mean(np.abs(targets_original[active_mask] - predictions_original[active_mask]))
            r2_active = r2_score(targets_original[active_mask], predictions_original[active_mask])
        else:
            mae_active = 0
            r2_active = 0
        
        # ====================================================================
        # IMPRIMIR RESULTADOS
        # ====================================================================
        print(f"\n{'='*80}")
        print(f"MÉTRICAS: {set_name}")
        print(f"{'='*80}")
        print(f"\n📊 MÉTRICAS GLOBALES:")
        print(f"  MAE:        {mae:>10.2f} m³")
        print(f"  RMSE:       {rmse:>10.2f} m³")
        print(f"  R²:         {r2:>10.4f}")
        print(f"  WMAPE:      {wmape:>10.2f} %")
        print(f"  SMAPE:      {smape:>10.2f} %")
        print(f"  MAPE (>10): {mape_10:>10.2f} %")
        
        print(f"\n📊 PRODUCCIÓN ACTIVA (> 10 m³):")
        print(f"  Registros: {active_count:,}")
        print(f"  MAE:       {mae_active:>10.2f} m³")
        print(f"  R²:        {r2_active:>10.4f}")
        
        # Métricas por variable
        pred_reshaped = predictions_original.reshape(S, H, N, F)
        target_reshaped = targets_original.reshape(S, H, N, F)
        
        print(f"\n📊 MÉTRICAS POR VARIABLE:")
        print(f"{'Variable':<20} {'MAE (m³)':<12} {'WMAPE (%)':<10} {'R²':<10}")
        print("-" * 60)
        
        metrics_by_var = {}
        
        for feat_idx, var_name in enumerate(target_names):
            pred_var = pred_reshaped[:, :, :, feat_idx].flatten()
            target_var = target_reshaped[:, :, :, feat_idx].flatten()
            
            mae_var = np.mean(np.abs(target_var - pred_var))
            wmape_var = (np.sum(np.abs(target_var - pred_var)) / 
                        (np.sum(np.abs(target_var)) + epsilon)) * 100
            
            ss_res_var = np.sum((target_var - pred_var) ** 2)
            ss_tot_var = np.sum((target_var - np.mean(target_var)) ** 2)
            r2_var = 1 - (ss_res_var / ss_tot_var) if ss_tot_var > 0 else 0
            
            print(f"{var_name:<20} {mae_var:<12.2f} {wmape_var:<10.2f} {r2_var:<10.4f}")
            
            metrics_by_var[var_name] = {
                'mae': mae_var,
                'wmape': wmape_var,
                'r2': r2_var
            }
        
        print(f"{'='*80}")
        
        return {
            'mae': mae,
            'mape': mape,
            'mape_10': mape_10,
            'smape': smape,
            'wmape': wmape,
            'rmse': rmse,
            'r2': r2,
            'mae_active': mae_active,
            'r2_active': r2_active,
            'active_count': active_count,
            'metrics_by_var': metrics_by_var,
            'predictions': predictions_original,
            'targets': targets_original
        }
    
    # ========================================================================
    # EVALUAR CADA CONJUNTO
    # ========================================================================
    
    print("\n" + "="*80)
    print("EVALUANDO TODOS LOS CONJUNTOS")
    print("="*80)
    
    train_metrics = evaluate_single_set(dm_norm.train_dataloader(), "TRAIN")
    val_metrics = evaluate_single_set(dm_norm.val_dataloader(), "VALIDATION")
    test_metrics = evaluate_single_set(dm_norm.test_dataloader(), "TEST")
    
    # ========================================================================
    # COMPARACIÓN FINAL
    # ========================================================================
    
    print("\n" + "="*80)
    print("COMPARACIÓN: TRAIN vs VAL vs TEST")
    print("="*80)
    
    print(f"\n{'Métrica':<15} {'TRAIN':<15} {'VAL':<15} {'TEST':<15}")
    print("-" * 65)
    
    metrics_to_compare = ['r2', 'mae', 'wmape', 'rmse']
    metric_names = ['R²', 'MAE (m³)', 'WMAPE (%)', 'RMSE (m³)']
    
    for metric_key, metric_name in zip(metrics_to_compare, metric_names):
        train_val = train_metrics[metric_key]
        val_val = val_metrics[metric_key]
        test_val = test_metrics[metric_key]
        print(f"{metric_name:<15} {train_val:<15.2f} {val_val:<15.2f} {test_val:<15.2f}")
    
    # ========================================================================
    # ANÁLISIS DE OVERFITTING
    # ========================================================================
    
    print("\n" + "="*80)
    print("ANÁLISIS DE OVERFITTING")
    print("="*80)
    
    train_r2 = train_metrics['r2']
    val_r2 = val_metrics['r2']
    test_r2 = test_metrics['r2']
    
    print(f"\nR² por conjunto:")
    print(f"  Train: {train_r2:.4f}")
    print(f"  Val:   {val_r2:.4f}")
    print(f"  Test:  {test_r2:.4f}")
    
    diff_train_val = train_r2 - val_r2
    diff_val_test = abs(val_r2 - test_r2)
    
    if diff_train_val > 0.10:
        print(f"\n⚠️  POSIBLE OVERFITTING:")
        print(f"   Diferencia Train-Val: {diff_train_val:.4f} (>0.10)")
        print(f"   El modelo se ajusta demasiado a los datos de entrenamiento")
    elif abs(diff_train_val) < 0.05:
        print(f"\n✅ BUEN BALANCE:")
        print(f"   Diferencia Train-Val: {abs(diff_train_val):.4f} (<0.05)")
        print(f"   El modelo generaliza bien")
    else:
        print(f"\n✅ ACEPTABLE:")
        print(f"   Diferencia Train-Val: {abs(diff_train_val):.4f}")
    
    if diff_val_test > 0.05:
        print(f"\n⚠️  Diferencia Val-Test: {diff_val_test:.4f}")
        print(f"   Posible variabilidad en distribución de datos")
    else:
        print(f"\n✅ Val y Test consistentes: diff={diff_val_test:.4f}")
    
    print("="*80)
    
    # ========================================================================
    # RETORNAR RESULTADOS
    # ========================================================================
    
    return {
        'train': train_metrics,
        'val': val_metrics,
        'test': test_metrics
    }



In [33]:
# Evaluar todos los conjuntos
all_metrics = evaluate_all_sets_manual(
    predictor=predictor,
    dm_norm=dm_norm,
    train_tensor=train_tensor,
    full_target=full_target,
    train_length=len(train_tensor),
    val_length=len(val_tensor)
)


# Verificar overfitting
train_val_diff = all_metrics['train']['r2'] - all_metrics['val']['r2']
if train_val_diff > 0.10:
    print("\n⚠️  Posible overfitting detectado")
else:
    print("\n✅ Sin overfitting significativo")

EVALUACIÓN COMPLETA: TRAIN, VAL y TEST

[0/5] Recreando scalers desde datos de train...
  Scaler 0 (Oil (m3)): mean=4.0149, std=2.7042
  Scaler 1 (Gas (m3)): mean=8.0968, std=5.4370
  Scaler 2 (Produced Water (m3)): mean=1.2803, std=2.0721
✓ Scalers recreados con datos de TRAIN

EVALUANDO TODOS LOS CONJUNTOS

EVALUANDO: TRAIN

[1/4] Obteniendo predicciones de TRAIN...
✓ Predicciones: torch.Size([4000, 7, 3, 3])
  Nota: Están en escala NORMALIZADA

[2/4] Desnormalizando...
✓ Desnormalizado (aún con log): [-1.84, 13.72]

[3/4] Invirtiendo log-transform (np.expm1)...
✓ Escala original (m³): [0.00, 908985.19]

[4/4] Calculando métricas...

MÉTRICAS: TRAIN

📊 MÉTRICAS GLOBALES:
  MAE:          18344.75 m³
  RMSE:         66701.10 m³
  R²:             0.7148
  WMAPE:           40.70 %
  SMAPE:           94.64 %
  MAPE (>10):     116.96 %

📊 PRODUCCIÓN ACTIVA (> 10 m³):
  Registros: 138,477
  MAE:         29599.78 m³
  R²:            0.7201

📊 MÉTRICAS POR VARIABLE:
Variable             MAE (

In [34]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path

# ========== 1. GUARDAR MODELO ENTRENADO ==========
def save_model_complete(predictor, feature_cols, well_mapping, edge_index_tensor, edge_weight, model_name="TimeThenSpace", save_dir="models"):
    """Guardar modelo completo con metadatos"""
    Path(save_dir).mkdir(exist_ok=True),
    
    # Guardar modelo completo
    torch.save({
        'model_state_dict': predictor.model.state_dict(),
        'model_class': type(predictor.model).__name__,
        'input_size': 8,
        'output_size': 3,  
        'n_nodes': 3,
        'horizon': 7,
        'feature_cols': feature_cols,
        'well_mapping': well_mapping,
        'edge_index': edge_index_tensor,  # ✅ Guardar grafo
        'edge_weight': edge_weight, 
        'model_params': {
            'hidden_size': 96,
            'rnn_layers': 3,
            'gnn_kernel': 5,
            'dropout': 0.1,
            'output_size':3
        }
    }, f"{save_dir}/{model_name}_kyle_complete.pt")
    
    print(f"Modelo {model_name} guardado en {save_dir}/{model_name}_kyle_complete.pt")


save_model_complete(
    predictor=predictor,
    feature_cols=feature_cols,
    well_mapping=well_mapping,
    edge_index_tensor=edge_index_tensor,
    edge_weight=edge_weight
)



Modelo TimeThenSpace guardado en models/TimeThenSpace_kyle_complete.pt


## SPACETHENTIME MODEL

In [ ]:
import torch.nn as nn
from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange
class SpaceThenTimeModel(nn.Module):
    def __init__(self, input_size, n_nodes, horizon, edge_index, edge_weight, 
                 hidden_size=64, rnn_layers=1, gnn_kernel=3, dropout=0.1, output_size=3):
        super(SpaceThenTimeModel, self).__init__()
        
        self.input_size = input_size      # 8
        self.output_size = output_size    # 3
        self.horizon = horizon            # 7
        self.n_nodes = n_nodes            # 4
        
        self.encoder = nn.Linear(input_size, hidden_size)
        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)
        
        # Primero espacio, después tiempo
        self.space_nn = DiffConv(
            in_channels=hidden_size,
            out_channels=hidden_size, 
            k=gnn_kernel
        )
        
        self.time_nn = RNN(
            input_size=hidden_size,
            hidden_size=hidden_size,
            n_layers=rnn_layers,
            cell='gru',
            return_only_last_state=True
        )
        
        self.decoder = nn.Linear(hidden_size, output_size * horizon)
        
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon, f=output_size)
        
        self.dropout = nn.Dropout(dropout)
        
        self.register_buffer('edge_index', edge_index)
        self.register_buffer('edge_weight', edge_weight)
    
    def forward(self, x):
    
        # Encoder: [B, W, N, 8] → [B, W, N, hidden]
        x_enc = self.encoder(x)
        
        # Add embeddings
        x_emb = x_enc + self.node_embeddings()
        
        # Procesar espacialmente cada timestep
        batch_size, seq_len, n_nodes, features = x_emb.shape
        outputs = []
        
        for t in range(seq_len):
            # Spatial: [B, N, hidden] → [B, N, hidden]
            h_t = self.space_nn(x_emb[:, t], self.edge_index, self.edge_weight)
            outputs.append(h_t)
        
        # Stack: [B, W, N, hidden]
        h_seq = torch.stack(outputs, dim=1)
        
        # Temporal: [B, W, N, hidden] → [B, N, hidden]
        h = self.time_nn(h_seq)
        
        # Dropout
        h = self.dropout(h)
        
        # Decoder: [B, N, hidden] → [B, N, horizon*output_size]
        # Ejemplo: [B, 4, 96] → [B, 4, 21]
        x_out = self.decoder(h)
        
        # Rearrange: [B, N, 21] → [B, 7, N, 3]
        x_horizon = self.rearrange(x_out)
        
        return x_horizon




In [ ]:
modelSTT = SpaceThenTimeModel(
    input_size=8,
    output_size=3,
    n_nodes=3,
    horizon=7,
    edge_index=edge_index_tensor,
    edge_weight=edge_weight,
    hidden_size=128,
    rnn_layers=1,
    gnn_kernel=2,
)
print(modelSTT)


SpaceThenTimeModel(
  (encoder): Linear(in_features=8, out_features=128, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=3, embedding_size=128)
  (space_nn): DiffConv(128, 128)
  (time_nn): RNN(
    (rnn): GRU(128, 128)
  )
  (decoder): Linear(in_features=128, out_features=21, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=7, f=3)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [ ]:
# Crear predictor con métricas
predictorSTT = Predictor(
    model=modelSTT,
    optim_class=torch.optim.AdamW,
    optim_kwargs={'lr': 0.0003, 'weight_decay': 0.005},
    loss_fn=MaskedMAE(),
    metrics={
        'mae': MaskedMAE(),
        'mse': MaskedMSE(),
        'r2': MaskedR2()
    }
)


# Checkpoint optimizado para Kyle Field
checkpoint_callbackSTT = ModelCheckpoint(
    dirpath='logs/kyle_field',
    save_top_k=1,
    monitor='val_mae',             
    mode='min',
    filename='kyle-spacetime-{epoch:02d}-{val_mae:.4f}',
    save_weights_only=True,
    verbose=False
)

# Early stopping para evitar overfitting
early_stoppingSTT = EarlyStopping(
    monitor='val_mae',              
    patience=15,                  
    min_delta=0.01,                 
    mode='min',
    verbose=True
)

# Trainer optimizado para Kyle Field
trainerSTT = pl.Trainer(
    max_epochs=100,                 
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    gradient_clip_val=1.0,        
    limit_train_batches=None,       
    val_check_interval=1.0,         
    log_every_n_steps=5,            
    enable_progress_bar=True,
    callbacks=[checkpoint_callbackSTT, early_stoppingSTT],
    deterministic=False,            
    benchmark=True                  
)

# Entrenamiento
print("Iniciando entrenamiento STT para Kyle Field...")
trainerSTT.fit(predictorSTT, datamodule=dm_norm)

# Evaluación final
test_resultsSTT = trainerSTT.test(predictorSTT, datamodule=dm_norm)
print("\nResultados finales SpaceThenTimeModel:")
for metric, value in test_resultsSTT[0].items():
    if 'mape' not in metric.lower():  
        print(f"{metric}: {value:.4f}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..

  | Name          | Type               | Params | Mode 
-------------------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0      | train
1 | train_metrics | MetricCollection   | 0      | train
2 | val_metrics   | MetricCollection   | 0      | train
3 | test_metrics  | MetricCollection   | 0      | train
4 | model         | SpaceThenTimeModel | 185 K  | train
-------------------------------------------------------------
185 K     Trainable params
0         Non-trainable params
185 K     Total params
0.741     Total estimated model params size (MB)
24        Modules in train mode
0         Modules in eval mode


Iniciando entrenamiento STT para Kyle Field...
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

Arguments ['edge_index', 'edge_weight'] are filtered out. Only args ['x'] are forwarded to the model (SpaceThenTimeModel).


Epoch 0: 100%|██████████| 125/125 [00:09<00:00, 12.63it/s, v_num=39, val_mae=0.276, val_mse=0.397, val_r2=0.647, train_mae=0.374, train_mse=0.415, train_r2=0.583]

Metric val_mae improved. New best score: 0.276


...

Epoch 4: 100%|██████████| 125/125 [00:11<00:00, 10.98it/s, v_num=39, val_mae=0.251, val_mse=0.391, val_r2=0.652, train_mae=0.236, train_mse=0.303, train_r2=0.695]

Metric val_mae improved by 0.011 >= min_delta = 0.01. New best score: 0.251


Epoch 19: 100%|██████████| 125/125 [00:14<00:00,  8.86it/s, v_num=39, val_mae=0.256, val_mse=0.376, val_r2=0.664, train_mae=0.215, train_mse=0.290, train_r2=0.708]

Monitored metric val_mae did not improve in the last 15 records. Best score: 0.251. Signaling Trainer to stop.


Testing DataLoader 0: 100%|██████████| 42/42 [00:01<00:00, 26.87it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.10311809927225113
        test_mae            0.10311811417341232
        test_mse            0.07588354498147964
         test_r2            0.9286361932754517
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Resultados finales SpaceThenTimeModel:
test_mae: 0.1031
test_mse: 0.0759
test_r2: 0.9286
test_loss: 0.1031


In [38]:
# Evaluar todos los conjuntos
all_metrics = evaluate_all_sets_manual(
    predictor=predictorSTT,
    dm_norm=dm_norm,
    train_tensor=train_tensor,
    full_target=full_target,
    train_length=len(train_tensor),
    val_length=len(val_tensor)
)


# Verificar overfitting
train_val_diff = all_metrics['train']['r2'] - all_metrics['val']['r2']
if train_val_diff > 0.10:
    print("\n⚠️  Posible overfitting detectado")
else:
    print("\n✅ Sin overfitting significativo")

EVALUACIÓN COMPLETA: TRAIN, VAL y TEST

[0/5] Recreando scalers desde datos de train...
  Scaler 0 (Oil (m3)): mean=4.0149, std=2.7042
  Scaler 1 (Gas (m3)): mean=8.0968, std=5.4370
  Scaler 2 (Produced Water (m3)): mean=1.2803, std=2.0721
✓ Scalers recreados con datos de TRAIN

EVALUANDO TODOS LOS CONJUNTOS

EVALUANDO: TRAIN

[1/4] Obteniendo predicciones de TRAIN...
✓ Predicciones: torch.Size([4000, 7, 3, 3])
  Nota: Están en escala NORMALIZADA

[2/4] Desnormalizando...
✓ Desnormalizado (aún con log): [-2.28, 14.26]

[3/4] Invirtiendo log-transform (np.expm1)...
✓ Escala original (m³): [0.00, 1562317.62]

[4/4] Calculando métricas...

MÉTRICAS: TRAIN

📊 MÉTRICAS GLOBALES:
  MAE:          18611.46 m³
  RMSE:         65749.33 m³
  R²:             0.7229
  WMAPE:           41.29 %
  SMAPE:           97.56 %
  MAPE (>10):     104.81 %

📊 PRODUCCIÓN ACTIVA (> 10 m³):
  Registros: 138,445
  MAE:         30730.90 m³
  R²:            0.7195

📊 MÉTRICAS POR VARIABLE:
Variable             MAE 

In [39]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path

# ========== 1. GUARDAR MODELO ENTRENADO ==========
def save_model_complete(predictor, feature_cols, well_mapping, edge_index_tensor, edge_weight, model_name="SpaceThenTime", save_dir="models"):
    """Guardar modelo completo con metadatos"""
    Path(save_dir).mkdir(exist_ok=True)
    
    # Guardar modelo completo
    torch.save({
        'model_state_dict': predictor.model.state_dict(),
        'model_class': type(predictor.model).__name__,
        'input_size': 8,
        'output_size': 3,
        'n_nodes': 3,
        'horizon': 7,
        'feature_cols': feature_cols,
        'well_mapping': well_mapping,
        'edge_index': edge_index_tensor,  # ✅ Guardar grafo
        'edge_weight': edge_weight, 
        'model_params': {
            'hidden_size': 128,
            'rnn_layers': 1,
            'gnn_kernel': 2,
            'output_size':3
        }
    }, f"{save_dir}/{model_name}_kyle_complete.pt")
    
    print(f"Modelo {model_name} guardado en {save_dir}/{model_name}_kyle_complete.pt")

# Guardar modelo actual
save_model_complete(
    predictor=predictorSTT,
    feature_cols=feature_cols,
    well_mapping=well_mapping,
    edge_index_tensor=edge_index_tensor,
    edge_weight=edge_weight
)





Modelo SpaceThenTime guardado en models/SpaceThenTime_kyle_complete.pt


## CRNN

In [ ]:
import torch
import torch.nn as nn
from tsl.nn.models import DCRNNModel
from einops.layers.torch import Rearrange

class SimpleDCRNNModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 edge_index, edge_weight,
                 hidden_size: int = 128,
                 rnn_layers: int = 3,
                 dropout: float = 0.1,
                 output_size: int = 3):
        super().__init__()
        
        self.input_size = input_size      # 8
        self.output_size = output_size    # 3
        self.horizon = horizon            # 7
        self.n_nodes = n_nodes            # 4
        
        self.dcrnn = DCRNNModel(
            input_size=input_size,
            hidden_size=hidden_size,
            output_size=input_size,  
            horizon=horizon,
            n_layers=rnn_layers
        )
        
  
        self.projection = nn.Linear(input_size, output_size)
        
        # Registrar topología del grafo
        self.register_buffer('edge_index', edge_index)
        self.register_buffer('edge_weight', edge_weight)
        
    def forward(self, x):
      
        # DCRNN: [B, 14, 4, 8] → [B, 7, 4, 8]
        out = self.dcrnn(x, edge_index=self.edge_index, edge_weight=self.edge_weight)
        
        # Proyección: [B, 7, 4, 8] → [B, 7, 4, 3]
        out = self.projection(out)
        
        return out

In [41]:
# Instanciar modelo

modelDCRNN = SimpleDCRNNModel(
    input_size=8,
    output_size=3,
    n_nodes=3,
    horizon=7,
    edge_index=edge_index_tensor,
    edge_weight=edge_weight,
    hidden_size=128,
    rnn_layers=3,
    dropout=0.1
)
print(modelDCRNN)

SimpleDCRNNModel(
  (dcrnn): DCRNNModel(
    (input_encoder): Linear(in_features=8, out_features=128, bias=True)
    (dcrnn): DCRNN(cell=DCRNNCell, return_only_last_state=True)
    (readout): MLPDecoder(
      (readout): MLP(
        (mlp): Sequential(
          (0): Dense(
            (affinity): Linear(in_features=128, out_features=256, bias=True)
            (activation): ReLU()
            (dropout): Identity()
          )
        )
        (readout): Linear(in_features=256, out_features=56, bias=True)
      )
      (rearrange): Rearrange('b n (h f) -> b h n f', f=8, h=7)
    )
  )
  (projection): Linear(in_features=8, out_features=3, bias=True)
)


In [ ]:

# Crear predictor con métricas
predictorDCRNN = Predictor(
    model=modelDCRNN,
    optim_class=torch.optim.AdamW,
    optim_kwargs={'lr': 0.0003, 'weight_decay': 0.0019},
    loss_fn=MaskedMAE(),
    metrics={
        'mae': MaskedMAE(),
        'mse': MaskedMSE(),
        'r2': MaskedR2()
    }
)


# Checkpoint optimizado para Kyle Field
checkpoint_callbackDCRNN = ModelCheckpoint(
    dirpath='logs/kyle_field',
    save_top_k=1,
    monitor='val_mae',            
    mode='min',
    filename='kyle-DCRNN-{epoch:02d}-{val_mae:.4f}',
    save_weights_only=False,
    verbose=True
)

# Early stopping para evitar overfitting
early_stoppingDCRNN = EarlyStopping(
    monitor='val_mae',            
    patience=15,                   
    min_delta=0.01,                
    mode='min',
    verbose=True
)

# Trainer optimizado para Kyle Field
trainerDCRNN = pl.Trainer(
    max_epochs=100,                 
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    gradient_clip_val=1.0,          
    limit_train_batches=None,       
    val_check_interval=1.0,         
    log_every_n_steps=5,           
    enable_progress_bar=True,
    callbacks=[checkpoint_callbackDCRNN, early_stoppingDCRNN],
    deterministic=False,            
    benchmark=True                  
)

# Entrenamiento
print("Iniciando entrenamiento DCRNN para Kyle Field...")
trainerDCRNN.fit(predictorDCRNN, datamodule=dm_norm)

# Evaluación final
test_resultsDCRNN = trainerDCRNN.test(predictorDCRNN, datamodule=dm_norm)
print("\nResultados finales TimeThenSpace:")
for metric, value in test_resultsDCRNN[0].items():
    if 'mape' not in metric.lower(): 
        print(f"{metric}: {value:.4f}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | loss_fn       | MaskedMAE        | 0      | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
3 | test_metrics  | MetricCollection | 0      | train
4 | model         | SimpleDCRNNModel | 1.5 M  | train
-----------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.097     Total estimated model params size (MB)
58        Modules in train mode
0         Modules in eval mode


Iniciando entrenamiento DCRNN para Kyle Field...
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

Arguments ['edge_index', 'edge_weight'] are filtered out. Only args ['x'] are forwarded to the model (SimpleDCRNNModel).


Epoch 0: 100%|██████████| 125/125 [01:17<00:00,  1.60it/s, v_num=40, val_mae=0.294, val_mse=0.446, val_r2=0.606, train_mae=0.463, train_mse=0.653, train_r2=0.343]

Metric val_mae improved. New best score: 0.294
Epoch 0, global step 125: 'val_mae' reached 0.29447 (best 0.29447), saving model to 'C:\\Users\\DIEGO\\OneDrive\\Escritorio\\Maestria Ciencia de datos\\Tesis\\Tests\\mape copy\\logs\\kyle_field\\kyle-DCRNN-epoch=00-val_mae=0.2945.ckpt' as top 1


...

Epoch 16: 100%|██████████| 125/125 [03:20<00:00,  0.62it/s, v_num=40, val_mae=0.255, val_mse=0.406, val_r2=0.638, train_mae=0.182, train_mse=0.277, train_r2=0.722]

Epoch 16, global step 2125: 'val_mae' was not in top 1


Epoch 17: 100%|██████████| 125/125 [04:50<00:00,  0.43it/s, v_num=40, val_mae=0.261, val_mse=0.418, val_r2=0.628, train_mae=0.179, train_mse=0.272, train_r2=0.726]

Monitored metric val_mae did not improve in the last 15 records. Best score: 0.260. Signaling Trainer to stop.
Epoch 17, global step 2250: 'val_mae' was not in top 1


Testing DataLoader 0: 100%|██████████| 42/42 [00:10<00:00,  4.20it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.1135929748415947
        test_mae            0.1135929748415947
        test_mse            0.09930799901485443
         test_r2            0.9038656949996948
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Resultados finales TimeThenSpace:
test_mae: 0.1136
test_mse: 0.0993
test_r2: 0.9039
test_loss: 0.1136


In [43]:
# Evaluar todos los conjuntos
all_metrics = evaluate_all_sets_manual(
    predictor=predictorDCRNN,
    dm_norm=dm_norm,
    train_tensor=train_tensor,
    full_target=full_target,
    train_length=len(train_tensor),
    val_length=len(val_tensor)
)


# Verificar overfitting
train_val_diff = all_metrics['train']['r2'] - all_metrics['val']['r2']
if train_val_diff > 0.10:
    print("\n⚠️  Posible overfitting detectado")
else:
    print("\n✅ Sin overfitting significativo")

EVALUACIÓN COMPLETA: TRAIN, VAL y TEST

[0/5] Recreando scalers desde datos de train...
  Scaler 0 (Oil (m3)): mean=4.0149, std=2.7042
  Scaler 1 (Gas (m3)): mean=8.0968, std=5.4370
  Scaler 2 (Produced Water (m3)): mean=1.2803, std=2.0721
✓ Scalers recreados con datos de TRAIN

EVALUANDO TODOS LOS CONJUNTOS

EVALUANDO: TRAIN

[1/4] Obteniendo predicciones de TRAIN...
✓ Predicciones: torch.Size([4000, 7, 3, 3])
  Nota: Están en escala NORMALIZADA

[2/4] Desnormalizando...
✓ Desnormalizado (aún con log): [-0.27, 13.81]

[3/4] Invirtiendo log-transform (np.expm1)...
✓ Escala original (m³): [0.00, 994842.69]

[4/4] Calculando métricas...

MÉTRICAS: TRAIN

📊 MÉTRICAS GLOBALES:
  MAE:          16171.31 m³
  RMSE:         62145.93 m³
  R²:             0.7520
  WMAPE:           35.89 %
  SMAPE:           82.77 %
  MAPE (>10):     116.25 %

📊 PRODUCCIÓN ACTIVA (> 10 m³):
  Registros: 138,490
  MAE:         26066.39 m³
  R²:            0.7584

📊 MÉTRICAS POR VARIABLE:
Variable             MAE (

In [44]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path

# ========== 1. GUARDAR MODELO ENTRENADO ==========
def save_model_complete(predictor,  feature_cols, well_mapping, edge_index_tensor, edge_weight,model_name="SimpleDCRNNModel", save_dir="models"):
    """Guardar modelo completo con metadatos"""
    Path(save_dir).mkdir(exist_ok=True)
    
    # Guardar modelo completo
    torch.save({
        'model_state_dict': predictor.model.state_dict(),
        'model_class': type(predictor.model).__name__,
        'input_size': 8,
        'n_nodes': 3,
        'horizon': 7,
        'feature_cols': feature_cols,
        'well_mapping': well_mapping,
        'edge_index': edge_index_tensor,  # ✅ Guardar grafo
        'edge_weight': edge_weight, 
        'model_params': {
            'hidden_size': 128,
            'rnn_layers': 3,
            'dropout': 0.1,
            'output_size':3
        }
    }, f"{save_dir}/{model_name}_kyle_complete.pt")
    
    print(f"Modelo {model_name} guardado en {save_dir}/{model_name}_kyle_complete.pt")



save_model_complete(
    predictor=predictorDCRNN,
    feature_cols=feature_cols,
    well_mapping=well_mapping,
    edge_index_tensor=edge_index_tensor,
    edge_weight=edge_weight
)





Modelo SimpleDCRNNModel guardado en models/SimpleDCRNNModel_kyle_complete.pt


## GatedGraphNetworkModel (libreria TSL)


In [45]:
import torch
import torch.nn as nn
from tsl.nn.models import GatedGraphNetworkModel

class GatedGraphModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 edge_index, edge_weight,
                 hidden_size: int = 128,
                 input_window_size: int = 14,
                 enc_layers: int = 1,
                 gnn_layers: int = 3,
                 output_size: int = 3):
        super().__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.horizon = horizon
        self.n_nodes = n_nodes
        
        self.gated_gnn = GatedGraphNetworkModel(
            input_size=input_size,
            input_window_size=dm_norm.window,
            horizon=horizon,
            n_nodes=n_nodes,
            hidden_size=hidden_size,
            output_size=input_size,
            exog_size=0,
            enc_layers=enc_layers,
            gnn_layers=gnn_layers,
            full_graph=True,
            activation='silu'
        )
        
        self.projection = nn.Linear(input_size, output_size)
        
        # Convertir edge_index: (tensor, tensor) -> tensor [2, num_edges]
        if isinstance(edge_index, tuple) and len(edge_index) == 2:
            edge_index = torch.stack(edge_index, dim=0)
        elif not isinstance(edge_index, torch.Tensor):
            edge_index = torch.tensor(edge_index, dtype=torch.long)
        else:
            edge_index = edge_index.long()
        
        # Convertir edge_weight
        if not isinstance(edge_weight, torch.Tensor):
            edge_weight = torch.tensor(edge_weight, dtype=torch.float32)
        else:
            edge_weight = edge_weight.float()
        
        self.register_buffer('edge_index', edge_index)
        self.register_buffer('edge_weight', edge_weight)
        
    def forward(self, x):
        out = self.gated_gnn(x, edge_index=self.edge_index)
        out = self.projection(out)
        return out



In [ ]:

input_window_size = 14  

modelggcm = GatedGraphModel(
    input_size=8,
    output_size=3,
    n_nodes=3,
    horizon=7,
    edge_index=edge_index,
    edge_weight=edge_weight,
    hidden_size=128,
    input_window_size=dm_norm.window, 
    enc_layers=1,  # 1 (mapear rnn_layers a enc_layers)
    gnn_layers=3,   # 2 (mapear gnn_kernel a gnn_layers)
)
print(modelggcm)

GatedGraphModel(
  (gated_gnn): GatedGraphNetworkModel(
    (input_encoder): Sequential(
      (0): Linear(in_features=112, out_features=128, bias=True)
    )
    (encoder_layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): SiLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
      )
    )
    (emb): NodeEmbedding(n_nodes=3, embedding_size=128)
    (gcn_layers): ModuleList(
      (0-2): 3 x GatedGraphNetwork(128, 128)
    )
    (decoder): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): SiLU()
    )
    (readout): Sequential(
      (0): Linear(in_features=128, out_features=56, bias=True)
      (1): Rearrange('b n (h f) -> b h n f', h=7, f=8)
    )
  )
  (projection): Linear(in_features=8, out_features=3, bias=True)
)


In [ ]:


predictorggcm = Predictor(
    model=modelggcm,                 
    optim_class=torch.optim.Adam,  
    optim_kwargs={
        'lr': 0.005,                        
        'weight_decay': 0.01,               
        'betas': (0.9, 0.999),              
        'eps': 1e-8
    },    
    loss_fn=MaskedMAE(),               
    metrics={
        'mae': MaskedMAE(),
        'mse': MaskedMSE(),
        'r2': MaskedR2()
    }              
)

checkpoint_callbackggcm = ModelCheckpoint(
    dirpath='logs/kyle_field',
    save_top_k=1,
    monitor='val_mae',         
    mode='min',
    filename='kyle-ggcn-{epoch:02d}-{val_mae:.4f}',
    save_weights_only=False,
    verbose=True
)

# Early stopping para evitar overfitting
early_stoppingggcm = EarlyStopping(
    monitor='val_mae',              
    patience=15,                   
    min_delta=0.01,                
    mode='min',
    verbose=True
)

# Trainer optimizado para Kyle Field
trainerggcm = pl.Trainer(
    max_epochs=100,                
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    gradient_clip_val=1.0,        
    limit_train_batches=None,       
    val_check_interval=1.0,         
    log_every_n_steps=5,            
    enable_progress_bar=True,
    callbacks=[checkpoint_callbackggcm, early_stoppingggcm],
    deterministic=False,            
    benchmark=True                  
)

# Entrenamiento
print("Iniciando entrenamiento GATEDGRAPH para Kyle Field...")
trainerggcm.fit(predictorggcm, datamodule=dm_norm)

# Evaluación final
test_resultsggcm = trainerggcm.test(predictorggcm, datamodule=dm_norm)
print("\nResultados finales GGCM:")
for metric, value in test_resultsggcm[0].items():
    if 'mape' not in metric.lower(): 
        print(f"{metric}: {value:.4f}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | loss_fn       | MaskedMAE        | 0      | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
3 | test_metrics  | MetricCollection | 0      | train
4 | model         | GatedGraphModel  | 294 K  | train
-----------------------------------------------------------
294 K     Trainable params
0         Non-trainable params
294 K     Total params
1.178     Total estimated model params size (MB)
76        Modules in train mode
0         Modules in eval mode


Iniciando entrenamiento GATEDGRAPH para Kyle Field...
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

Arguments ['edge_index', 'edge_weight'] are filtered out. Only args ['x'] are forwarded to the model (GatedGraphModel).


Epoch 0: 100%|██████████| 125/125 [00:03<00:00, 32.20it/s, v_num=41, val_mae=0.326, val_mse=0.404, val_r2=0.641, train_mae=0.305, train_mse=0.351, train_r2=0.647]

Metric val_mae improved. New best score: 0.326
Epoch 0, global step 125: 'val_mae' reached 0.32597 (best 0.32597), saving model to 'C:\\Users\\DIEGO\\OneDrive\\Escritorio\\Maestria Ciencia de datos\\Tesis\\Tests\\mape copy\\logs\\kyle_field\\kyle-ggcn-epoch=00-val_mae=0.3260.ckpt' as top 1


...

Epoch 23: 100%|██████████| 125/125 [00:04<00:00, 29.80it/s, v_num=41, val_mae=0.270, val_mse=0.400, val_r2=0.645, train_mae=0.228, train_mse=0.313, train_r2=0.685]

Epoch 23, global step 3000: 'val_mae' was not in top 1


Epoch 24: 100%|██████████| 125/125 [00:04<00:00, 30.25it/s, v_num=41, val_mae=0.268, val_mse=0.401, val_r2=0.643, train_mae=0.227, train_mse=0.312, train_r2=0.686]

Monitored metric val_mae did not improve in the last 15 records. Best score: 0.263. Signaling Trainer to stop.
Epoch 24, global step 3125: 'val_mae' was not in top 1


Testing DataLoader 0: 100%|██████████| 42/42 [00:00<00:00, 79.17it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.09131509065628052
        test_mae            0.09131508320569992
        test_mse            0.07423687726259232
         test_r2            0.9302377104759216
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Resultados finales GGCM:
test_mae: 0.0913
test_mse: 0.0742
test_r2: 0.9302
test_loss: 0.0913


In [48]:
# Evaluar todos los conjuntos
all_metrics = evaluate_all_sets_manual(
    predictor=predictorggcm,
    dm_norm=dm_norm,
    train_tensor=train_tensor,
    full_target=full_target,
    train_length=len(train_tensor),
    val_length=len(val_tensor)
)

# Acceder a métricas individuales
print("\n📊 RESUMEN GATED GRAPH NETWORK:")
print(f"\nTRAIN:")
print(f"  R²:    {all_metrics['train']['r2']:.4f}")
print(f"  WMAPE: {all_metrics['train']['wmape']:.2f}%")

print(f"\nVAL:")
print(f"  R²:    {all_metrics['val']['r2']:.4f}")
print(f"  WMAPE: {all_metrics['val']['wmape']:.2f}%")

print(f"\nTEST:")
print(f"  R²:    {all_metrics['test']['r2']:.4f}")
print(f"  WMAPE: {all_metrics['test']['wmape']:.2f}%")

# Verificar overfitting
train_val_diff = all_metrics['train']['r2'] - all_metrics['val']['r2']
if train_val_diff > 0.10:
    print("\n⚠️  Posible overfitting detectado")
else:
    print("\n✅ Sin overfitting significativo")

EVALUACIÓN COMPLETA: TRAIN, VAL y TEST

[0/5] Recreando scalers desde datos de train...
  Scaler 0 (Oil (m3)): mean=4.0149, std=2.7042
  Scaler 1 (Gas (m3)): mean=8.0968, std=5.4370
  Scaler 2 (Produced Water (m3)): mean=1.2803, std=2.0721
✓ Scalers recreados con datos de TRAIN

EVALUANDO TODOS LOS CONJUNTOS

EVALUANDO: TRAIN

[1/4] Obteniendo predicciones de TRAIN...
✓ Predicciones: torch.Size([4000, 7, 3, 3])
  Nota: Están en escala NORMALIZADA

[2/4] Desnormalizando...
✓ Desnormalizado (aún con log): [-3.52, 14.20]

[3/4] Invirtiendo log-transform (np.expm1)...
✓ Escala original (m³): [0.00, 1468402.12]

[4/4] Calculando métricas...

MÉTRICAS: TRAIN

📊 MÉTRICAS GLOBALES:
  MAE:          22472.57 m³
  RMSE:         78350.50 m³
  R²:             0.6061
  WMAPE:           49.91 %
  SMAPE:           65.64 %
  MAPE (>10):     114.61 %

📊 PRODUCCIÓN ACTIVA (> 10 m³):
  Registros: 138,419
  MAE:         37484.33 m³
  R²:            0.5895

📊 MÉTRICAS POR VARIABLE:
Variable             MAE 

In [49]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path

# ========== 1. GUARDAR MODELO ENTRENADO ==========
def save_model_complete(predictor,  feature_cols, well_mapping, edge_index_tensor, edge_weight,model_name="GatedGraphModel", save_dir="models"):
    """Guardar modelo completo con metadatos"""
    Path(save_dir).mkdir(exist_ok=True)
    
    # Guardar modelo completo
    torch.save({
        'model_state_dict': predictor.model.state_dict(),
        'model_class': type(predictor.model).__name__,
        'input_size': 8,
        'n_nodes': 3,
        'horizon': 7,
        'feature_cols': feature_cols,
        'well_mapping': well_mapping,
        'edge_index': edge_index_tensor,  # ✅ Guardar grafo
        'edge_weight': edge_weight, 
        'model_params': {
            'input_size':8,
            'input_window_size': 14,
            'n_nodes':3,
            'horizon':7,
            'edge_index': edge_index,
            'edge_weight': edge_weight,
            'hidden_size':128,
            'enc_layers':1,  # 1 (mapear rnn_layers a enc_layers)
            'gnn_layers':3,   # 2 (mapear gnn_kernel a gnn_layers)
            'output_size' : 3
        }
    }, f"{save_dir}/{model_name}_kyle_complete.pt")
    
    print(f"Modelo {model_name} guardado en {save_dir}/{model_name}_kyle_complete.pt")

# Guardar modelo actual
save_model_complete(
    predictor=predictorggcm,
    feature_cols=feature_cols,
    well_mapping=well_mapping,
    edge_index_tensor=edge_index_tensor,
    edge_weight=edge_weight
)


Modelo GatedGraphModel guardado en models/GatedGraphModel_kyle_complete.pt
